In [754]:
import numpy as np
import math
import random
import cPickle
import collections
import nltk
import scipy.stats
import scipy as sp
from nltk.corpus import stopwords
from textblob import TextBlob as tb
import collections
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
#=========================test full vocabulary==============================

In [87]:
# load vocabulary.txt and store it in an array
fv=open('/Users/weiweili/Documents/UCSD-Stats/CSE 250B Fall16/HW2/vocabulary.txt','r')
vb=[]
for line in fv.readlines():
    b=line.strip()
    vb.append(b)
    
fv.close()

vb_num=len(vb)
print vb_num



61188


In [5]:
# load tain.label
frl=open('/Users/weiweili/Documents/UCSD-Stats/CSE 250B Fall16/HW2/20news-bydate/matlab/train.label','r')
trn_l=[]
for line in frl.readlines():
    b=int(line.strip())
    trn_l.append(b)
    
frl.close()
    
doc_num=len(trn_l)
print doc_num
#print trn_l

11269


In [6]:
# load test.label
fsl=open('/Users/weiweili/Documents/UCSD-Stats/CSE 250B Fall16/HW2/20news-bydate/matlab/test.label','r')
tst_l=[]
for line in fsl.readlines():
    b=int(line.strip())
    tst_l.append(b)
fsl.close()
print len(tst_l)
#print tst_l


7505


In [7]:
# load train.data
frd=open('/Users/weiweili/Documents/UCSD-Stats/CSE 250B Fall16/HW2/20news-bydate/matlab/train.data','r')
trn_matrix=np.zeros([doc_num,vb_num])
for line in frd.readlines():
    b = line.strip().split()     #train.data formatted as "docIdx wordIdx count", b[0]=docIdx; b[1]=wordIdx; b[2]=count
    if(len(b) != 3):
        print "data missing"
    trn_matrix[int(b[0])-1][int(b[1])-1] = int(b[2])  #b[0]=docIdx: 1,2,3,....
frd.close()

print trn_matrix.shape
print trn_matrix[11268][53950:53958]

(11269, 61188)
[ 0.  0.  0.  0.  0.  0.  0.  1.]


In [8]:
# load test.data
tst_matrix=np.zeros([len(tst_l),vb_num])
fsd=open('/Users/weiweili/Documents/UCSD-Stats/CSE 250B Fall16/HW2/20news-bydate/matlab/test.data','r')
for line in fsd.readlines():
    b = line.strip().split()
    if (len(b) !=3):
        print "data missing"
    tst_matrix[int(b[0])-1][int(b[1])-1]=int(b[2])
        
fsd.close()

print tst_matrix.shape
print tst_matrix[7504]

(7505, 61188)
[ 0.  0.  0. ...,  0.  0.  2.]


In [10]:
# n_j: 20*2 the matrix of the sum of words in class j
nj=collections.defaultdict(int)
for i in range (doc_num):
    if trn_l[i] in nj:         #doc_num = 11269, only 20 classes
        nj[trn_l[i]] += np.sum(trn_matrix[i]) 
    else:
        nj[trn_l[i]] = np.sum(trn_matrix[i])
        
print nj.items()
# nj[key]: key=1,2,...,20

[(1, 148812.0), (2, 110358.0), (3, 90767.0), (4, 99146.0), (5, 86190.0), (6, 152846.0), (7, 61094.0), (8, 114102.0), (9, 102631.0), (10, 107898.0), (11, 141267.0), (12, 200456.0), (13, 103173.0), (14, 155338.0), (15, 153714.0), (16, 201267.0), (17, 175914.0), (18, 254805.0), (19, 186426.0), (20, 119096.0)]


In [11]:
# n_{jw}: 20*vb_num #the matrix of the number of word 'w' in class j
K = 20
n_jw = np.zeros([K, vb_num])
for i in range(doc_num):
    for j in range(vb_num):
        if i % 500 == 0 and j == (vb_num-1):    
            print i,j
        n_jw[trn_l[i]-1][j] += trn_matrix[i][j]                                      

0 61187
500 61187
1000 61187
1500 61187
2000 61187
2500 61187
3000 61187
3500 61187
4000 61187
4500 61187
5000 61187
5500 61187
6000 61187
6500 61187
7000 61187
7500 61187
8000 61187
8500 61187
9000 61187
9500 61187
10000 61187
10500 61187
11000 61187


In [14]:
# hj: the probability of each class
hj=collections.defaultdict(int)
for i in range(doc_num):
    if trn_l[i] in hj:
        hj[trn_l[i]] += 1
    else:
        hj[trn_l[i]] = 1
print hj
s=0
for key in hj:
    hj[key]= hj[key]*1.0/doc_num
    
    s +=hj[key]   #check if the sum is 0
print s,hj

defaultdict(<type 'int'>, {1: 480, 2: 581, 3: 572, 4: 587, 5: 575, 6: 592, 7: 582, 8: 592, 9: 596, 10: 594, 11: 598, 12: 594, 13: 591, 14: 594, 15: 593, 16: 599, 17: 545, 18: 564, 19: 464, 20: 376})
1.0 defaultdict(<type 'int'>, {1: 0.04259472890229834, 2: 0.05155736977549028, 3: 0.05075871860857219, 4: 0.05208980388676901, 5: 0.051024935664211554, 6: 0.052533498979501284, 7: 0.051646108794036735, 8: 0.052533498979501284, 9: 0.052888455053687104, 10: 0.0527109770165942, 11: 0.05306593309078002, 12: 0.0527109770165942, 13: 0.05244475996095483, 14: 0.0527109770165942, 15: 0.052622237998047744, 16: 0.05315467210932647, 17: 0.04836276510781791, 18: 0.05004880646020055, 19: 0.04117490460555506, 20: 0.033365870973467035})


In [15]:
log_hj=collections.defaultdict(int)
for key in hj:
    log_hj[key] = math.log(hj[key])
    #print log_hj[key]

-3.15602476805
-2.9650601151
-2.98067188057
-2.95478605212
-2.97544083115
-2.94630423707
-2.96334042422
-2.94630423707
-2.93957020489
-2.94293155259
-2.936220118
-2.94293155259
-2.94799485455
-2.94293155259
-2.94461647295
-2.93454927384
-3.02902507729
-2.99475662045
-3.18992631973
-3.40022172856


In [16]:
#calculate P_jw with smoothing
p_jw=np.zeros([K,vb_num])
for j in range(K):
    for w in range(vb_num):
        p_jw[j][w]=(n_jw[j][w]+1)*1.0/(nj[j+1]+vb_num)

In [18]:
# use logs to avoid underflow
log_p_jw=np.zeros([K,vb_num])
for j in range(K):
    for w in range(vb_num):
        log_p_jw[j][w]=math.log(p_jw[j][w])

In [18]:
# test
correct_classified = 0
for i in range(len(tst_l)):
    predict = 0
    max_val = float('-Inf')
    for k in range(K):
        tmp = log_hj[k+1]
        tmp += np.dot(tst_matrix[i], log_p_jw[k])  #sum of log(P_ki ^ xi)
        #for j in range(V):
        #    tmp += test_matrix[i][j] * log_prob[k][j]
        if tmp > max_val:
            max_val = tmp
            predict = k+1
    if predict == tst_l[i]:
        correct_classified += 1
print 'the number of correctly classified:', correct_classified
print "error rate: ", 1 - correct_classified * 1.0 / len(tst_l)

the number of correctly classified: 5862
error rate:  0.21892071952


In [23]:
#===========================================================================

In [110]:
#save results
cPickle.dump(hj, open('hj.pkl','wb'))
cPickle.dump(p_jw, open('p_jw.pkl','wb'))
cPickle.dump(log_hj, open('log_hj.pkl','wb'))
cPickle.dump(log_p_jw, open('log_p_jw.pkl','wb'))

In [ ]:
#================================reduced_voca withou stopwords====================

In [19]:
from nltk.corpus import stopwords
import nltk; nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/weiweili/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [181]:
#nltk.download()

In [352]:
# reduce vocabulary by reducing stopwords
reduced_voca = []
stopwords = stopwords.words('english')
fv=open('/Users/weiweili/Documents/UCSD-Stats/CSE 250B Fall16/HW2/vocabulary.txt','r')
for line in fv.readlines():
    b = line.strip()
    if not b in stopwords:
        reduced_voca.append(b)
fv.close()
num_rvb= len(reduced_voca)
print num_rvb

61044


In [353]:
vb_index=np.zeros([num_rvb])
for j in range(num_rvb):
    vb_index[j]= vb.index(reduced_voca[j])

In [354]:
# test matrix for reduced vocab
n_tst_matrix = np.zeros([len(tst_l),num_rvb])
for i in range(len(tst_l)):
    for j in range(num_rvb):
        if i % 500 == 0 and j == (num_rvb-1):    
            print i,j
        n_tst_matrix[i][j] = tst_matrix[i][int(vb_index[j])]
        

0 61043
500 61043
1000 61043
1500 61043
2000 61043
2500 61043
3000 61043
3500 61043
4000 61043
4500 61043
5000 61043
5500 61043
6000 61043
6500 61043
7000 61043
7500 61043


In [38]:
# train matrix for reduced vocab
n_trn_matrix = np.zeros([len(trn_l),num_rvb])
for i in range(len(trn_l)):
    for j in range(num_rvb):
        if i % 500 == 0 and j == (num_rvb-1):    
            print i,j
        n_trn_matrix[i][j] = trn_matrix[i][int(vb_index[j])]
  

0 61043
500 61043
1000 61043
1500 61043
2000 61043
2500 61043
3000 61043
3500 61043
4000 61043
4500 61043
5000 61043
5500 61043
6000 61043
6500 61043
7000 61043
7500 61043
8000 61043
8500 61043
9000 61043
9500 61043
10000 61043
10500 61043
11000 61043


In [357]:
# n_j for reduced vocab: 20*20 the matrix of the sum of words in class j
nj=collections.defaultdict(int)
for i in range (doc_num):
    if trn_l[i] in nj:
        nj[trn_l[i]] += np.sum(n_trn_matrix[i]) 
    else:
        nj[trn_l[i]] = np.sum(n_trn_matrix[i])
        
print nj.items()
# nj[key]: key=1,2,...,20

[(1, 78618.0), (2, 69489.0), (3, 56845.0), (4, 59876.0), (5, 50837.0), (6, 96306.0), (7, 41757.0), (8, 64991.0), (9, 61026.0), (10, 63397.0), (11, 88599.0), (12, 116774.0), (13, 60943.0), (14, 91403.0), (15, 94858.0), (16, 102430.0), (17, 98629.0), (18, 137147.0), (19, 101027.0), (20, 63228.0)]


In [359]:
# n_{jw}: 20*vb_num #the matrix of the number of word 'w' in class j
K = 20
n_jw = np.zeros([K, num_rvb])
for i in range(doc_num):
    for j in range(num_rvb):
        if i % 500 == 0 and j == (num_rvb-1):    
            print i,j
        n_jw[trn_l[i]-1][j] += n_trn_matrix[i][j]         

0 61043
500 61043
1000 61043
1500 61043
2000 61043
2500 61043
3000 61043
3500 61043
4000 61043
4500 61043
5000 61043
5500 61043
6000 61043
6500 61043
7000 61043
7500 61043
8000 61043
8500 61043
9000 61043
9500 61043
10000 61043
10500 61043
11000 61043


In [355]:
#calculate P_jw with smoothing
p_jw=np.zeros([K,num_rvb])
for j in range(K):
    for w in range(num_rvb):
        p_jw[j][w]=(n_jw[j][w]+1)*1.0/(nj[j+1]+num_rvb)

In [364]:
# use logs to avoid underflow
log_p_jw=np.zeros([K,num_rvb])
for j in range(K):
    for w in range(num_rvb):
        log_p_jw[j][w]=math.log(p_jw[j][w])

In [365]:
print log_p_jw.shape

(20, 61044)


In [368]:
# test
correct_classified = 0
for i in range(len(tst_l)):
    predict = 0
    max_val = float('-Inf')
    for k in range(K):
        tmp = log_hj[k+1]
        tmp += np.dot(n_tst_matrix[i], log_p_jw[k])  #sum of log(P_jw ^ xi)
        if tmp > max_val:
            max_val = tmp
            predict = k+1
    if predict == tst_l[i]:
        correct_classified += 1
print 'the number of correctly classified:', correct_classified
print "error rate: ", 1 - correct_classified * 1.0 / len(tst_l)

the number of correctly classified: 6010
error rate:  0.199200532978


In [ ]:
#================Calculate CI for Random selection from reduced_voca===============

In [369]:
#calculate CI:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    c, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return c,h

In [753]:
#random selectin from reduced voca 
M=[5000,10000,20000]
CI=[]
for m in M:
    rd_error=[]
    for n in range(50):
        rsvb=np.random.choice(reduced_voca,m)
        rsvb_index=np.zeros([m])
        for j in range(m):
            rsvb_index[j]= reduced_voca.index(rsvb[j])
        
        
        # train matrix
        rn_trn_matrix = np.zeros([len(trn_l),m])
        for i in range(len(trn_l)):
            for j in range(m):
                rn_trn_matrix[i][j] = n_trn_matrix[i][int(rsvb_index[j])]
        
        
        #test matrix        
        rn_tst_matrix = np.zeros([len(tst_l),m])
        for i in range(len(tst_l)):
            for j in range(m):
                rn_tst_matrix[i][j] = n_tst_matrix[i][int(rsvb_index[j])]
        
        
        #n_j
        nj=collections.defaultdict(int)
        for i in range (doc_num):
            if trn_l[i] in nj:
                nj[trn_l[i]] += np.sum(rn_trn_matrix[i]) 
            else:
                nj[trn_l[i]] = np.sum(rn_trn_matrix[i])
                
                    
        # n_{jw}: 20*m #the matrix of the number of word 'w' in class j
        K = 20
        n_jw = np.zeros([K, m])
        for i in range(doc_num):
            for j in range(m):
                n_jw[trn_l[i]-1][j] += rn_trn_matrix[i][j]    
                
                
        #calculate P_jw with smoothing
        K=20
        p_jw=np.zeros([K,m])
        for j in range(K):
            for w in range(m):
                p_jw[j][w]=(n_jw[j][w]+1)*1.0/(nj[j+1]+m)
    
                
        # use logs to avoid underflow
        log_p_jw=np.zeros([K,m])
        for j in range(K):
            for w in range(m):
                log_p_jw[j][w]=math.log(p_jw[j][w])
        
        # test for random selctected reduced_voca
        correct_classified = 0
        for i in range(len(tst_l)):
            predict = 0
            max_val = float('-Inf')
            for k in range(K):
                tmp = log_hj[k+1]
                tmp += np.dot(rn_tst_matrix[i], log_p_jw[k])  #sum of log(P_jw ^ xi)

                if tmp > max_val:
                    max_val = tmp
                    predict = k+1
            if predict == tst_l[i]:
                correct_classified += 1
        #print 'the number of correctly classified:', correct_classified
        error=1 - correct_classified * 1.0 / len(tst_l)
        rd_error.append(error)
        print n
    print m
    c=mean_confidence_interval(rd_error, confidence=0.95 )[0]
    h=mean_confidence_interval(rd_error, confidence=0.95 )[1]
    CI.append([c,h])
print CI

0
5000
[[0.56735509660226513, nan]]


In [ ]:
#==================random selection from reduced_voca======================

In [371]:
#randomly select m from reduced_voca
m=5000
rsvb=np.random.choice(reduced_voca,m)
rsvb_index=np.zeros([m])
for j in range(m):
    rsvb_index[j]= reduced_voca.index(rsvb[j])
print rsvb_index.shape
print rsvb_index[0:50]

(5000,)
[ 13295.  29734.  23505.  36039.  40832.  47367.  44614.  15639.  22546.
  59945.  11607.  48054.  48503.  56802.   7452.  26099.  56990.  19989.
  58506.  41856.   5379.  39821.  19252.   7531.   3695.  23951.  56580.
  35072.  30576.  45669.  44770.  36507.  59384.  14880.   6040.  22134.
  24218.   6326.   9790.  39316.  38262.  53462.  11652.  34809.   1669.
  41261.  31459.  11369.   9401.  40138.]


In [372]:
# train matrix for reduced vocab
rn_trn_matrix = np.zeros([len(trn_l),m])
for i in range(len(trn_l)):
    for j in range(m):
        if i % 500 == 0 and j == (m-1):    
            print i,j
        rn_trn_matrix[i][j] = n_trn_matrix[i][int(rsvb_index[j])]
        

0 4999
500 4999
1000 4999
1500 4999
2000 4999
2500 4999
3000 4999
3500 4999
4000 4999
4500 4999
5000 4999
5500 4999
6000 4999
6500 4999
7000 4999
7500 4999
8000 4999
8500 4999
9000 4999
9500 4999
10000 4999
10500 4999
11000 4999


In [376]:
## test matrix for random selceted reduced vocab
rn_tst_matrix = np.zeros([len(tst_l),m])
for i in range(len(tst_l)):
    for j in range(m):
        if i % 500 == 0 and j == (m-1):    
            print i,j
        rn_tst_matrix[i][j] = n_tst_matrix[i][int(rsvb_index[j])]
        
        

0 4999
500 4999
1000 4999
1500 4999
2000 4999
2500 4999
3000 4999
3500 4999
4000 4999
4500 4999
5000 4999
5500 4999
6000 4999
6500 4999
7000 4999
7500 4999


In [378]:
# n_j for random selected reduced vocab: 20*20 the matrix of the sum of words in class j
nj=collections.defaultdict(int)
for i in range (doc_num):
    if trn_l[i] in nj:
        nj[trn_l[i]] += np.sum(rn_trn_matrix[i]) 
    else:
        nj[trn_l[i]] = np.sum(rn_trn_matrix[i])
        
print nj.items()
# nj[key]: key=1,2,...,20

[(1, 5631.0), (2, 5111.0), (3, 4592.0), (4, 4409.0), (5, 3942.0), (6, 7727.0), (7, 3298.0), (8, 5354.0), (9, 4358.0), (10, 4403.0), (11, 6077.0), (12, 9465.0), (13, 4226.0), (14, 6979.0), (15, 7273.0), (16, 7522.0), (17, 7947.0), (18, 10609.0), (19, 7220.0), (20, 4675.0)]


In [435]:
# n_{jw}: 20*m #the matrix of the number of word 'w' in class j
K = 20
n_jw = np.zeros([K, m])
for i in range(doc_num):
    for j in range(m):
        if i % 500 == 0 and j == (m-1):    
            print i,j
        n_jw[trn_l[i]-1][j] += rn_trn_matrix[i][j]    

0 4999
500 4999
1000 4999
1500 4999
2000 4999
2500 4999
3000 4999
3500 4999
4000 4999
4500 4999
5000 4999
5500 4999
6000 4999
6500 4999
7000 4999
7500 4999
8000 4999
8500 4999
9000 4999
9500 4999
10000 4999
10500 4999
11000 4999


In [436]:
#calculate P_jw with smoothing
K=20 
p_jw=np.zeros([K,m])
for j in range(K):
    for w in range(m):
        p_jw[j][w]=(n_jw[j][w]+1)*1.0/(nj[j+1]+m)

In [438]:
# use logs to avoid underflow
log_p_jw=np.zeros([K,m])
for j in range(K):
    for w in range(m):
        log_p_jw[j][w]=math.log(p_jw[j][w])

In [445]:
# test for random selctected reduced_voca
correct_classified = 0
for i in range(len(tst_l)):
    predict = 0
    max_val = float('-Inf')
    for k in range(K):
        tmp = log_hj[k+1]
        tmp += np.dot(rn_tst_matrix[i], log_p_jw[k])  #sum of log(P_jw ^ xi)
        
        if tmp > max_val:
            max_val = tmp
            predict = k+1
    
    if predict == tst_l[i]:
        correct_classified += 1
  
print "the number of correctly classified:", correct_classified
print "error rate: ", 1 - correct_classified * 1.0 / len(tst_l)

the number of correctly classified: 3408
error rate:  0.545902731512


In [ ]:
#=============================vocab select by tf-idf=============================

In [ ]:
#==========use formula for Tfidf=========

In [841]:
# n_{jw}: 20*vb_num #the matrix of the number of word 'w' in class j
K = 20
n_jw = np.zeros([K, num_rvb])
for i in range(doc_num):
    for j in range(num_rvb):
        if i % 500 == 0 and j == (num_rvb-1):    
            print i,j
        n_jw[trn_l[i]-1][j] += n_trn_matrix[i][j]         

0 61043
500 61043
1000 61043
1500 61043
2000 61043
2500 61043
3000 61043
3500 61043
4000 61043
4500 61043
5000 61043
5500 61043
6000 61043
6500 61043
7000 61043
7500 61043
8000 61043
8500 61043
9000 61043
9500 61043
10000 61043
10500 61043
11000 61043


In [842]:
transformer = TfidfTransformer()  
X = transformer.fit_transform(n_jw)
tfidf_matrix=X.toarray()     #20x61044

In [812]:
print len(tfidf_matrix[:,0])


20


In [ ]:
#==========================================

In [843]:
# get the max tfidf for each word in 20 classes
max_tfidf=[]
for i in range (num_rvb):
    max_tfidf.append(max(tfidf_matrix[:,i]))

In [760]:
print len(max_tfidf)

61044


In [869]:
m=20000

sorted_words = sorted(enumerate(max_tfidf), key=lambda x: x[1], reverse=True)[0:m]
sorted_index=[item[0] for item in sorted_words]


In [871]:
print len(sorted_index)


20000


In [872]:
# train matrix
v_trn_matrix = np.zeros([len(trn_l),m])
for i in range(len(trn_l)):
    for j in range(m):
        if i % 500 == 0 and j == (m-1):    
            print i,j
        v_trn_matrix[i][j] = n_trn_matrix[i][int(sorted_index[j])]
        

0 19999
500 19999
1000 19999
1500 19999
2000 19999
2500 19999
3000 19999
3500 19999
4000 19999
4500 19999
5000 19999
5500 19999
6000 19999
6500 19999
7000 19999
7500 19999
8000 19999
8500 19999
9000 19999
9500 19999
10000 19999
10500 19999
11000 19999


In [873]:
#test matrix        
v_tst_matrix = np.zeros([len(tst_l),m])
for i in range(len(tst_l)):
    for j in range(m):
        if i % 500 == 0 and j == (m-1):    
            print i,j

        v_tst_matrix[i][j] = n_tst_matrix[i][int(sorted_index[j])]
        

0 19999
500 19999
1000 19999
1500 19999
2000 19999
2500 19999
3000 19999
3500 19999
4000 19999
4500 19999
5000 19999
5500 19999
6000 19999
6500 19999
7000 19999
7500 19999


In [874]:
#n_j
nj=collections.defaultdict(int)
for i in range (doc_num):
    if trn_l[i] in nj:
        nj[trn_l[i]] += np.sum(v_trn_matrix[i]) 
    else:
        nj[trn_l[i]] = np.sum(v_trn_matrix[i])

In [729]:
print nj

defaultdict(<type 'int'>, {1: 50015.0, 2: 62286.0, 3: 41724.0, 4: 45388.0, 5: 38151.0, 6: 70904.0, 7: 27968.0, 8: 41521.0, 9: 36316.0, 10: 37746.0, 11: 47181.0, 12: 75682.0, 13: 40290.0, 14: 52747.0, 15: 59992.0, 16: 62464.0, 17: 58214.0, 18: 72076.0, 19: 61039.0, 20: 37183.0})


In [875]:
# n_{jw}: 20*m #the matrix of the number of word 'w' in class j
K = 20
n_jw = np.zeros([K, m])
for i in range(doc_num):
    for j in range(m):
        if i % 500 == 0 and j == (m-1):    
            print i,j
        n_jw[trn_l[i]-1][j] += v_trn_matrix[i][j]    

0 19999
500 19999
1000 19999
1500 19999
2000 19999
2500 19999
3000 19999
3500 19999
4000 19999
4500 19999
5000 19999
5500 19999
6000 19999
6500 19999
7000 19999
7500 19999
8000 19999
8500 19999
9000 19999
9500 19999
10000 19999
10500 19999
11000 19999


In [876]:
#calculate P_jw with smoothing
K=20
p_jw=np.zeros([K,m])
for j in range(K):
    for w in range(m):
        p_jw[j][w]=(n_jw[j][w]+1)*1.0/(nj[j+1]+m)

In [877]:
# use logs to avoid underflow
log_p_jw=np.zeros([K,m])
for j in range(K):
    for w in range(m):
        log_p_jw[j][w]=math.log(p_jw[j][w])

In [878]:
# test for tf-idf selctected reduced_voca
correct_classified = 0
for i in range(len(tst_l)):
    predict = 0
    max_val = float('-Inf')
    for k in range(K):
        tmp = log_hj[k+1]
        tmp += np.dot(v_tst_matrix[i], log_p_jw[k])  #sum of log(P_jw ^ xi)

        if tmp > max_val:
            max_val = tmp
            predict = k+1
    #print "predict",predict
    if predict == tst_l[i]:
        correct_classified += 1
    #print "correct", correct_classified
print 'the number of correctly classified:', correct_classified
print 1 - correct_classified * 1.0 / len(tst_l)
        


the number of correctly classified: 6009
0.199333777482


In [880]:
# find the largest M TF-IDF words in each class
toplabels=np.zeros((20,20))
topwords=[[None]*20 for i in range(20)]
for i in range(20):
    #find the largest 20 elements in tfidf_matrix(20x 1000)
    toplabels[i]=np.asarray(np.argpartition(-tfidf_matrix[i],20)[:20]) 
    for j in range(20):
        topwords[i][j]=reduced_voca[int(toplabels[i][j])]

In [881]:
for i in range(20):
    print topwords[i]


['people', 'com', 'atheism', 'writes', 'edu', 'one', 'article', 'god', 'would', 'atheists', 'islam', 'think', 'religion', 'say', 'livesey', 'like', 'morality', 'know', 'atheist', 'jesus']
['files', 'images', 'image', 'also', 'edu', 'writes', 'software', 'would', 'file', 'graphics', 'use', 'jpeg', 'ftp', 'gif', 'one', 'data', 'com', 'format', 'know', 'like']
['use', 'files', 'com', 'card', 'know', 'problem', 'win', 'dos', 'ei', 'file', 'like', 'one', 'windows', 'article', 'mouse', 'using', 'edu', 'writes', 'get', 'would']
['edu', 'one', 'bios', 'scsi', 'controller', 'ide', 'card', 'drive', 'com', 'mb', 'dos', 'disk', 'drives', 'system', 'bus', 'would', 'get', 'pc', 'use', 'floppy']
['apple', 'edu', 'mac', 'scsi', 'one', 'quadra', 'mb', 'simms', 'would', 'drive', 'nubus', 'writes', 'know', 'fpu', 'mhz', 'problem', 'centris', 'duo', 'get', 'use']
['edu', 'widget', 'com', 'window', 'motif', 'file', 'xterm', 'use', 'program', 'server', 'get', 'output', 'lib', 'contrib', 'oname', 'entry', 'x